In [1]:
# Set up PATH for imports
import sys
if ".." not in sys.path:
    sys.path.append("..")

# Introduction to the *R. pom* FBA model

This notebook provides a guide to loading, running, querying, and making temporary modifications to the *R. pom* FBA model.
We use the COBRApy package extensively, and in fact the *R. pom* model is stored in SBML format and always run through the COBRApy package.
As such, while I will cover some basic COBRApy concepts here, further questions about model manipulation should be referred to the [COBRApy docs](https://cobrapy.readthedocs.io/en/latest/).

Contents:
1. [Loading and Inspecting](#loading-and-inspecting)
2. [Model Structure](#model-structure)
    - [Reactions, Metabolites, and Genes](#reactions-metabolites-and-genes)
    - [Medium, Exchanges, Special Reactions](#medium-exchanges-special-reactions)
3. [Running and Querying](#running-and-querying)
    - [Useful patterns](#useful-patterns)
4. [Bonus Chapter: Debugging an Essential Reaction](#bonus-chapter-debugging-an-essential-reaction)

# Loading and Inspecting

The ready-to-use version of the *R. pom* model is stored in SBML format at path `model/Rpom_05.xml`. We can simply load using the `cobra.io.read_sbml_model` function.

(Note: there's also several versions of the model contained in the `base_model` directory - these are baseline models that the final model is built off of in an iterative fashion. There's no need to concern ourselves with those now however, as making permanent changes and rebuilding the model is a topic for another time.)

In [2]:
import cobra
import pandas as pd
from cobra.io import read_sbml_model

model = read_sbml_model("../model/Rpom_05.xml")
model

Name,Rpom_05
Memory address,11e055640
Number of metabolites,1797
Number of reactions,1796
Number of genes,968
Number of groups,0
Objective expression,1.0*Rpom_hwa_biomass - 1.0*Rpom_hwa_biomass_reverse_5ec2f
Compartments,"c, p, e"


# Model Structure
## Reactions, Metabolites, and Genes

An COBRApy model is largely defined by three key lists of interacting objects - namely, the reactions, metabolites, and genes:

In [3]:
print(f"Model has {len(model.reactions)} reactions, {len(model.metabolites)} metabolites, and {len(model.genes)} genes")

Model has 1796 reactions, 1797 metabolites, and 968 genes


Inspecting a few items in each list is illuminating as to their relationships. In brief, `Reaction`s have a stoichiometry, i.e., a dictionary of `Metabolite`s with their associated coefficients, representing how much of each metabolite is consumed or produced for one unit of flux. (Behind the scenes, these stoichiometric dictionaries are used by COBRApy to assemble the stoichiometric $S$ matrix.) `Gene`s relate to `Reaction`s through "Gene-Protein-Reaction" (GPR) rules, which specify a boolean relationship that determines when a `Reaction` is allowed to permit flux, as a function of whether the corresponding genes have been knocked out. 

As an example, below I access the reaction for glucose-6-phosphate isomerase (ID: `PGLUCISOM-RXN`, in accordance with [BioCyc](https://biocyc.org/reaction?orgid=GCF_000011965&id=PGLUCISOM-RXN)). I show how to inspect its stoichiometry, GPR rule, and flux bounds.

### Example: `Reaction` object

In [4]:
# Typically, we access reactions by their ID, which is a string.
# Almost all reactions in this model follow BioCyc conventions.
g6p_isomerase = model.reactions.get_by_id("PGLUCISOM-RXN")
g6p_isomerase

Reaction identifier,PGLUCISOM-RXN
Name,PGLUCISOM-RXN
Memory address,0x1505f9c40
Stoichiometry,D-glucopyranose-6-phosphate[c] <=> FRUCTOSE-6P[c] D-glucopyranose 6-phosphate <=> beta-D-fructofuranose 6-phosphate
GPR,SPO2046
Lower bound,-1000.0
Upper bound,1000.0


In [5]:
# Stoichiometry is stored in the `metabolites` attribute of a `Reaction`, as a dictionary mapping `Metabolite` objects to their stoichiometric coefficients.
g6p_isomerase.metabolites

{<Metabolite D-glucopyranose-6-phosphate[c] at 0x11e0edd00>: -1.0,
 <Metabolite FRUCTOSE-6P[c] at 0x11e088fa0>: 1.0}

In [6]:
# As a convenience, a human-friendly readable version of this stoichiometry can be accessed with the `reaction` attribute.
# This is especially helpful later on when querying large sets of reactions, as a way to see what reactions are doing at a glance.
g6p_isomerase.reaction

'D-glucopyranose-6-phosphate[c] <=> FRUCTOSE-6P[c]'

In [7]:
# GPR is stored in the `gene_reaction_rule` attribute as a string - here, just one gene.
# In general, this can be a complex Boolean expression, with "and" and "or" operators.
g6p_isomerase.gene_reaction_rule

'SPO2046'

In [9]:
# Flat set of genes associated with this reaction, as `Gene` objects.
g6p_isomerase.genes

frozenset({<Gene SPO2046 at 0x1396e6070>})

In [10]:
# Flux bounds are stored in the `lower_bound`, `upper_bound`, and `bounds` attributes.
# Later on, we'll edit bounds of some reactions - this can theoretically be done through any of these attributes,
# but because of a quirk of how the `lower_bound` and `upper_bound` attributes are implemented, it's better to just assign both at once using `bounds`.
#
# (If you're curious, the issue is that at no point is `lower_bound` permitted to be higher than `upper_bound`,
# or `upper_bound` permitted to be less than `lower_bound` - this raises an error. This can cause unexpected issues when
# trying to set both bounds in sequence - if the first set operation is invalid, an error is raised, and the second set operation will never be reached.)
print(f"lower bound: {g6p_isomerase.lower_bound}")
print(f"upper bound: {g6p_isomerase.upper_bound}")
print(f"bounds tuple: {g6p_isomerase.bounds}")

lower bound: -1000.0
upper bound: 1000.0
bounds tuple: (-1000.0, 1000.0)


As a *R. pom* model-specific note, I've put in many potentially helpful explanatory details on most `Reaction`, `Metabolite`, and `Gene` objects, stored in the `.notes` attribute. For reactions, these include:

- `EC Number`: (best guess at) EC numbers
- `Kegg ID`: corresponding ID in KEGG database
- `stem`: prefix of the reaction ID that should match with BioCyc, since some reactions have non-biocyc IDs due to "specialization" (replacement of generic metabolite classes with specific metabolite instances)
- `pathways`: BIOCYC IDs for metabolic pathways that the reaction participates in.
- `Gene Source`: where the reaction came from (currently just lists if a reaction was added manually)

In [11]:
# Notes for g6p isomerase
g6p_isomerase.notes

{'EC Number': '5.3.1.9',
 'Kegg ID': 'R00771',
 'stem': 'PGLUCISOM-RXN',
 'pathways': '[&apos;ANAGLYCOLYSIS-PWY&apos;, &apos;UDPNAGSYN-PWY&apos;, &apos;GLUCONEO-PWY&apos;]'}

Global assessment of how many reactions are augmented with each possible note:

In [12]:
from operator import add
from functools import reduce
from collections import Counter

reduce(add, [Counter(rxn.notes.keys()) for rxn in model.reactions])

Counter({'EC Number': 946,
         'Kegg ID': 757,
         'Gene Source': 183,
         'stem': 1285,
         'pathways': 1796,
         'source': 507})

### Example: `Metabolite`

`Metabolite` objects themselves have a lot of information about the metabolite, including its name, formula, charge, and more.

In [13]:
g6p = model.metabolites.get_by_id("D-glucopyranose-6-phosphate[c]")
g6p

Metabolite identifier,D-glucopyranose-6-phosphate[c]
Name,D-glucopyranose 6-phosphate
Memory address,0x11e0edd00
Formula,C6H11O9P
Compartment,c
In 5 reaction(s),"PHOSPHOGLUCMUT-RXN, GLUCOKIN-RXN, MYO-INOSITOL-1-PHOSPHATE-SYNTHASE-RXN, PGLUCISOM-RXN, GLU6PDEHYDROG-RXN"


In [14]:
# Human-readable name - present on some but not all metabolites
g6p.name

'D-glucopyranose 6-phosphate'

In [15]:
g6p.formula

'C6H11O9P'

In [16]:
g6p.elements

{'C': 6, 'H': 11, 'O': 9, 'P': 1}

In [17]:
g6p.charge

-2

Similarly to how `Reaction`s are linked to `Metabolite`s through their `.metabolites` attribute, all `Metabolite`s in the model refer to the reactions they participate in through the `reactions` attribute:

In [18]:
for rxn in g6p.reactions:
    print(f"{rxn.id}\n\t{rxn.reaction}\n")

PHOSPHOGLUCMUT-RXN
	GLC-1-P[c] <=> D-glucopyranose-6-phosphate[c]

GLUCOKIN-RXN
	ATP[c] + Glucopyranose[c] --> ADP[c] + D-glucopyranose-6-phosphate[c] + PROTON[c]

MYO-INOSITOL-1-PHOSPHATE-SYNTHASE-RXN
	D-glucopyranose-6-phosphate[c] --> 1-L-MYO-INOSITOL-1-P[c]

PGLUCISOM-RXN
	D-glucopyranose-6-phosphate[c] <=> FRUCTOSE-6P[c]

GLU6PDEHYDROG-RXN
	D-glucopyranose-6-phosphate[c] + NADP[c] --> D-6-P-GLUCONO-DELTA-LACTONE[c] + NADPH[c] + PROTON[c]



Notice that all metabolite IDs are suffixed by `[c]`, `[p]`, or `[e]` - these are compartment identifiers that tell us if the metabolite is in the `[c]`ytoplasm, `[p]`eriplasm, or `[e]xternal` space. The compartment code can also be accessed through the `.compartment` attribute:

In [19]:
g6p.compartment

'c'

In [20]:
Counter([met.compartment for met in model.metabolites])

Counter({'c': 1627, 'p': 98, 'e': 72})

Just like `Reaction`s, the `Metabolite`s are also augmented with notes - in this case, just KEGG IDs:

In [21]:
g6p.notes

{'Kegg ID': 'C00092'}

In [22]:
reduce(add, [Counter(met.notes.keys()) for met in model.metabolites])

Counter({'Kegg ID': 886})

### Example: `Gene`

Finally, `Gene` objects also store some relevant information. Some (but not many in the model) have a human readable name under `.name`, and they also refer to the reactions they participate in through the `.reactions` attribute.

In [23]:
pgi = list(g6p_isomerase.genes)[0]
pgi

Gene identifier,SPO2046
Name,G_SPO2046
Memory address,0x1396e6070
Functional,True
In 1 reaction(s),PGLUCISOM-RXN


In [24]:
pgi.reactions

frozenset({<Reaction PGLUCISOM-RXN at 0x1505f9c40>})

More importantly, genes can be knocked out, and their effects on associated reactions are automatically calculated according to the GPR's.
We've yet to cover how to run the model, but here's a preview:

In [25]:
with model:
    # Enable glucose uptake at 5.44 mmol/gDW/hr, which is the experimentally measured uptake rate for this model.
    model.reactions.get_by_id("EX_glc").bounds = (-5.44, 0)

    # Get baseline growth rate
    baseline = model.optimize()
    print(f"Baseline growth rate: {baseline.objective_value:.4f} 1/hr")

    # Get growth rate with pgi knocked out
    pgi.knock_out()
    pgi_ko = model.optimize()
    print(f"Growth rate with pgi knocked out: {pgi_ko.objective_value:.4f} 1/hr")

Baseline growth rate: 0.5680 1/hr
Growth rate with pgi knocked out: 0.5680 1/hr


## Medium, Exchanges, Special Reactions

Now that we've covered the basic attributes of `Reaction`, `Metabolite`, and `Gene` objects, we ought to make a few remarks on some special classes of reactions.

The first are the so-called "exchange" reactions, which allow flux of metabolites in/out of the model. By convention, these are defined such that negative flux is the direction of "producing" metabolites (influx), and positive flux is the direction of "consuming" metabolites (efflux).

These can be accessed through the `.exchanges` attribute on the model - notice that all the reactions here only concern `[e]` metabolites:

In [26]:
for rxn in model.exchanges:
    print(f"{rxn.id}: \t{rxn.reaction}\t{rxn.bounds}")

EX_CH4S: 	CPD-7671[e] --> 	(0.0, 1000.0)
EX_ac: 	ACET[e] --> 	(0.0, 1000.0)
EX_acgam: 	N-acetyl-D-glucosamine[e] --> 	(0.0, 1000.0)
EX_ascb: 	ASCORBATE[e] --> 	(0.0, 1000.0)
EX_ca2: 	CA+2[e] <=> 	(-1000.0, 1000.0)
EX_cbl: 	CPD-315[e] --> 	(0.0, 1000.0)
EX_chitob: 	CHITOBIOSE[e] --> 	(0.0, 1000.0)
EX_cl: 	CL-[e] <=> 	(-1000.0, 1000.0)
EX_co2: 	CARBON-DIOXIDE[e] <=> 	(-1000.0, 1000.0)
EX_cobalt2: 	CO+2[e] <=> 	(-1000.0, 1000.0)
EX_cu2: 	CU+2[e] <=> 	(-1000.0, 1000.0)
EX_dmsp: 	SS-DIMETHYL-BETA-PROPIOTHETIN[e] --> 	(0.0, 1000.0)
EX_ectoine: 	ECTOINE[e] --> 	(0.0, 1000.0)
EX_fe2: 	FE+2[e] <=> 	(-1000.0, 1000.0)
EX_fe3: 	FE+3[e] <=> 	(-1000.0, 1000.0)
EX_fru: 	BETA-D-FRUCTOSE[e] --> 	(0.0, 1000.0)
EX_g6p: 	D-glucopyranose-6-phosphate[e] --> 	(0.0, 1000.0)
EX_gal: 	GALACTOSE[e] --> 	(0.0, 1000.0)
EX_gam: 	GLUCOSAMINE[e] --> 	(0.0, 1000.0)
EX_glt: 	GLT[e] --> 	(0.0, 1000.0)
EX_glyc: 	GLYCEROL[e] --> 	(0.0, 1000.0)
EX_glycolate: 	GLYCOLATE[e] --> 	(0.0, 1000.0)
EX_h: 	PROTON[e] <=> 	(-1000.0, 

The set of exchange reactions that supports flux in the producing direction is important, since this effectively specifies the composition of the growth medium. This subset of reactions can be accessed through the `.medium` attribtue as a dictionary, with keys being the reactions and values being the absolute value of maximal flux allowed in the producing direction.

In [27]:
model.medium

{'EX_ca2': 1000.0,
 'EX_cl': 1000.0,
 'EX_co2': 1000.0,
 'EX_cobalt2': 1000.0,
 'EX_cu2': 1000.0,
 'EX_fe2': 1000.0,
 'EX_fe3': 1000.0,
 'EX_h': 1000.0,
 'EX_h2o': 1000.0,
 'EX_k': 1000.0,
 'EX_mg2': 1000.0,
 'EX_mn2': 1000.0,
 'EX_mobd': 1000.0,
 'EX_na1': 1000.0,
 'EX_nh4': 1000.0,
 'EX_ni2': 1000.0,
 'EX_o2': 20.0,
 'EX_pi': 1000.0,
 'EX_slnt': 1000.0,
 'EX_so4': 1000.0,
 'EX_thm': 1000.0,
 'EX_zn2': 1000.0,
 'EX_BIOTIN[e]': 1000.0,
 'EX_NITRATE[e]': 1000.0,
 'EX_HCO3[e]': 1000.0}

Note that the medium is defined by default without a carbon source, so that the model does not grow by default. We have to specify a carbon source uptake rate to allow for growth.

Two other "special" reactions are of particular note: the Biomass reaction, and the non-growth-associated ATP maintenance flux.

The Biomass reaction is the objective of the FBA optimization problem. Unlike other reactions, whose stoichiometric coefficients are in units of mmol and fluxes are in units of mmol / g dry cell weight / hour, the biomass reaction has coeffients of g cell weight / mmol, such that the units of its flux are 1 / hr. The scaling of the coefficients is such that **the flux of the biomass reaction is interpreted as the growth rate of the bacterium itself**!

In [28]:
# Biomass reaction for our model.
# Note that this is a "pseudo-reaction" - it doesn't represent a real biochemical reaction, but rather a sink for all the biomass precursors.
# The stoichiometry of this reaction is determined by the biomass composition of the organism, which is typically measured experimentally.
# Note that this reaction is defined in terms of "pseudo-metabolites" that represent the biomass precursors,
# such as "PROTEIN[c]" and "RNA[c]". Inspecting the reactions associated with these pseudo-metabolites
# reveals further details about the composition of these biomass precursors, e.g. the ratio of different amino acids in the protein biomass precursor.
model.reactions.get_by_id("Rpom_hwa_biomass").reaction

'31.36179486 ATP[c] + 0.001524574 COFACTORS[c] + 0.05136379 DNA[c] + 0.009825009 IONS[c] + 0.095950926 LIPID[c] + 0.026355529 MUREIN[c] + 0.000447 NADP[c] + 0.001831 NAD[c] + 0.091372857 PHB-STORAGE[c] + 0.561521914 PROTEIN[c] + 0.093597842 RNA[c] + 48.601527 WATER[c] --> 31.36179573 ADP[c] + 31.36179573 PROTON[c] + 31.35745773 Pi[c]'

In [ ]:
with model:
    model.reactions.get_by_id("EX_glc").bounds = (-5.44, 0)
    solution = model.optimize()

print(f"Objective value: {solution.objective_value:.4f} 1/hr")
print(f"Biomass flux: {solution.fluxes['Rpom_hwa_biomass']:.4f} 1/hr")

Objective value: 0.5680 1/hr
Biomass flux: 0.5680 1/hr


The (non-growth-associated) ATP maintenance cost is a constant flux of ATP that is forced to be burned. The motivation for this is that several sources of energy usage are not accounted for purely by the metabolic activities of the cell - for instance, the operation of the flagella requires ATP for each rotation, but that isn't accounted for in FBA.

By default, the ATP maintenance flux is currently set to 0, but later on the intention is to pick a value that best fits experimental observations.

In [30]:
atpm = model.reactions.get_by_id("ATPM")
atpm

Reaction identifier,ATPM
Name,ATPM
Memory address,0x1503c2d00
Stoichiometry,ATP[c] + WATER[c] --> ADP[c] + PROTON[c] + Pi[c] ATP + H2O --> ADP + H+ + phosphate
GPR,
Lower bound,0.0
Upper bound,0.0


# Running and Querying

At this point, we've run the model a few times above, but a few more explanatory notes are in order.

First is the use of the `with` context block. When using the `with` statement, ***all changes made to the model within the block are automatically reverted upon exiting the block!***

I **highly** recommend using this statement any time you're making temporary changes to the model - otherwise, your changes in a given code block will compound, and you may find yourself facing unexpected bugs. This includes things like editing bounds, knocking out genes, or temporarily adding reactions to test growth rates under given conditions.

In [31]:
with model:
    print(">> Entered context manager")

    # Set glucose uptake bounds and verify that they are set correctly.
    model.reactions.get_by_id("EX_glc").bounds = (-5.44, 0)
    print(f"EX_glc bounds: {model.reactions.get_by_id('EX_glc').bounds}")

    # Knock out pgi and verify that it is knocked out.
    pgi.knock_out()
    print(f"PGI functional?: {pgi.functional}")

    # Add an example reaction and verify that it is added.
    test_rxn = cobra.Reaction("TEST_RXN")
    test_rxn.add_metabolites({g6p: -1})
    model.add_reactions([test_rxn])
    print(f"TEST_RXN in model?: {'TEST_RXN' in model.reactions}\n")

    solution = model.optimize()

print(">> Left context manager")
print(f"EX_glc bounds: {model.reactions.get_by_id('EX_glc').bounds}")
print(f"PGI functional?: {pgi.functional}")
print(f"TEST_RXN in model?: {'TEST_RXN' in model.reactions}\n")

>> Entered context manager
EX_glc bounds: (-5.44, 0)
PGI functional?: False
TEST_RXN in model?: True

>> Left context manager
EX_glc bounds: (0.0, 1000.0)
PGI functional?: True
TEST_RXN in model?: False



As also noted above, by default the medium is defined without a carbon source. You'll want to set a carbon source uptake rate before optimizing, usually using glucose and the experimental rate of 5.44 mmol/gDCW/hr.

## Useful patterns

Finally, I'll close by giving a few examples of helpful query expressions. Getting used to navigating the model can be daunting, given the almost-2000 reactions!

List comprehensions will be your friend here, and it's a good idea to cross-reference with the two BioCyc databases and the KEGG database to get a better picture of where a given reaction sits within metabolic pathways.

There's no particular order to the following examples - just some cases that I've found come up often enough that they may be worth studying.

In [32]:
# Reactions in which a given metabolite participates.
# This is just the standard `reactions` attribute of a `Metabolite`, but it's important to remember!
g6p.reactions

frozenset({<Reaction GLU6PDEHYDROG-RXN at 0x1504aeeb0>,
           <Reaction GLUCOKIN-RXN at 0x150b1fc10>,
           <Reaction MYO-INOSITOL-1-PHOSPHATE-SYNTHASE-RXN at 0x1505adb20>,
           <Reaction PGLUCISOM-RXN at 0x1505f9c40>,
           <Reaction PHOSPHOGLUCMUT-RXN at 0x150607400>})

In [33]:
# Metabolites of a reaction.
# This is just the standard `metabolites` attribute of a `Reaction`, but it's important to remember!
list(model.reactions.get_by_id("PGLUCISOM-RXN").metabolites.keys())

[<Metabolite D-glucopyranose-6-phosphate[c] at 0x11e0edd00>,
 <Metabolite FRUCTOSE-6P[c] at 0x11e088fa0>]

In [34]:
# Suppose I don't quite know the exact ID of a reaction, but I know it has "glucose" in the name. I can search for it like this:
[rxn for rxn in model.reactions if "glucose" in rxn.id.lower()]

[<Reaction GLUCOSE-6-PHOSPHATE-1-EPIMERASE-RXN at 0x150b72ac0>]

In [35]:
# Another similar example - maybe I want to find metabolites that contain phosphorus:
[met for met in model.metabolites if "P" in met.elements][:5]

[<Metabolite ACYL-SN-GLYCEROL-3P[c] at 0x11e06f610>,
 <Metabolite L-PHOSPHATIDATE[c] at 0x11e06f790>,
 <Metabolite 2-METHYL-3-HYDROXY-BUTYRYL-COA[c] at 0x11e06f670>,
 <Metabolite 2-METHYL-ACETO-ACETYL-COA[c] at 0x11e06f760>,
 <Metabolite NADH[c] at 0x11e06f7f0>]

In [36]:
# As a slightly more complex variation, maybe I want to find exchange reactions that can be used to bring phosphorus into the system:
[rxn for rxn in model.exchanges if any("P" in met.elements for met in rxn.metabolites)] 

[<Reaction EX_cbl at 0x15047c7c0>,
 <Reaction EX_g6p at 0x1504849a0>,
 <Reaction EX_pi at 0x150493820>,
 <Reaction EX_GLYCEROL-3P at 0x150ab9b80>]

In [37]:
# Reactions that consume a given metabolite in a given FBA solution - 
# note that this is for a specific set of realized fluxes,
# not just the potential for a reaction to consume the metabolite based on its stoichiometry.
pyruvate = model.metabolites.get_by_id("PYRUVATE[c]")

# Get Solution object
with model:
    model.reactions.get_by_id("EX_glc").bounds = (-5.44, 0)
    solution = model.optimize()

# Filter for reactions with where the product of the flux and the stoichiometric coefficient of pyruvate is negative - i.e. reactions that consume pyruvate.
consuming_reactions = [rxn for rxn in pyruvate.reactions
                       if solution.fluxes[rxn.id] * rxn.metabolites[pyruvate] < 0]
for rxn in consuming_reactions:
    print(f"{rxn.id}\n\t{rxn.reaction}\n\tFlux: {solution.fluxes[rxn.id]:.4f} mmol/gDCW/hr\n")

ACETOLACTSYN-RXN
	PROTON[c] + 2.0 PYRUVATE[c] --> 2-ACETO-LACTATE[c] + CARBON-DIOXIDE[c]
	Flux: 0.4127 mmol/gDCW/hr

DXS-RXN
	GAP[c] + PROTON[c] + PYRUVATE[c] --> CARBON-DIOXIDE[c] + DEOXYXYLULOSE-5P[c]
	Flux: 0.0015 mmol/gDCW/hr

D-ALANINE-AMINOTRANSFERASE-RXN
	2-KETOGLUTARATE[c] + D-ALANINE[c] <=> D-GLT[c] + PYRUVATE[c]
	Flux: -0.0006 mmol/gDCW/hr

PYRUVDEH-RXN
	CO-A[c] + NAD[c] + PYRUVATE[c] --> ACETYL-COA[c] + CARBON-DIOXIDE[c] + NADH[c]
	Flux: 5.4190 mmol/gDCW/hr

ACETOOHBUTSYN-RXN
	2-OXOBUTANOATE[c] + PROTON[c] + PYRUVATE[c] --> 2-ACETO-2-HYDROXY-BUTYRATE[c] + CARBON-DIOXIDE[c]
	Flux: 0.1371 mmol/gDCW/hr

DIHYDRODIPICSYN-RXN
	L-ASPARTATE-SEMIALDEHYDE[c] + PYRUVATE[c] --> CPD-14443[c] + PROTON[c] + WATER[c]
	Flux: 0.1624 mmol/gDCW/hr



In [38]:
# Reactions that change in flux following some perturbation - e.g. following a reaction knockout.
with model:
    model.reactions.get_by_id("EX_glc").bounds = (-5.44, 0)
    
    # Baseline solution with no knockouts.
    baseline_solution = model.optimize()

    # Knock out a reaction and get the new solution.
    model.reactions.get_by_id("D-ALANINE-AMINOTRANSFERASE-RXN").knock_out()
    ko_solution = model.optimize()

# Fluxes are stored as a pandas Series, which makes it easy to do operations,
# then export the results to csv or other formats!
baseline_fluxes = baseline_solution.fluxes
ko_fluxes = ko_solution.fluxes

# Get differences in fluxes, filter for significant changes, and sort by magnitude of change.
flux_changes = ko_fluxes - baseline_fluxes
significant_flux_changes = flux_changes[flux_changes.abs() > 1e-6]
significant_flux_changes = significant_flux_changes.sort_values(key=abs, ascending=False)
significant_flux_changes = significant_flux_changes.reset_index()
significant_flux_changes

# Some export options:
# ==========================================================================
# significant_flux_changes.to_clipboard(index=False)
# significant_flux_changes.to_csv("../results/flux_changes.csv", index=False)

,index,fluxes
0,MALATE-DEHYDROGENASE-NADP+-RXN,0.001801
1,METHENYLTHFCYCLOHYDRO-RXN,-0.000892
2,METHYLENETHFDEHYDROG-NADP-RXN,-0.000892
3,FUMHYDR-RXN,-0.000886
4,FORMATETHFLIG-RXN,0.000883
...,...,...
253,RXN-1381-PALMITYL-COA/GLYCEROL-3P//1-PALMITOYL...,-0.000001
254,CDPDIGLYSYN-RXN-2,-0.000001
255,DNA-RXN,-0.000001
256,ANTHRANSYN-RXN,-0.000001


# Bonus Chapter: Debugging an essential reaction

Here's an example of figuring out why a reaction is essential.Let's look at the following essential reaction:

In [44]:
tptase = model.reactions.get_by_id("THIAMIN-PYROPHOSPHOKINASE-RXN")
print(f"{tptase.id}:\n\t{tptase.reaction}\n")


with model:
    model.reactions.get_by_id("EX_glc").bounds = (-5.44, 0)

    sol = model.optimize()
    print(f"Growth rate with THIAMIN-PYROPHOSPHOKINASE-RXN: {sol.objective_value:.4f} 1/hr")
    print(f"Flux through tptase: {sol.fluxes[tptase.id]}\n")

    tptase.knock_out()

    sol = model.optimize()
    print(f"Growth rate without THIAMIN-PYROPHOSPHOKINASE-RXN: {sol.objective_value:.4f} 1/hr")

THIAMIN-PYROPHOSPHOKINASE-RXN:
	ATP[c] + THIAMINE[c] --> AMP[c] + PROTON[c] + THIAMINE-PYROPHOSPHATE[c]

Growth rate with THIAMIN-PYROPHOSPHOKINASE-RXN: 0.5680 1/hr
Flux through tptase: 0.00012665498825421742

Growth rate without THIAMIN-PYROPHOSPHOKINASE-RXN: 0.0000 1/hr


Looking at the reaction, some (potentially overlapping) hypotheses come to mind. I'll list them here in order of plausibility:

1. Without this reaction, not enough thiamine pyrophosphate is being produced for some downstream purpose.
2. Without this reaction, not enough thiamine is being consumed (i.e., the cell is stoichiometrically forced to produce thiamine at some rate higher than it is capable of consuming it, violating the steady-state assumption.)
3. Without this reaction, not enough AMP or H+ is being produced, or not enough ATP is being consumed. Unlikely since these are currency metabolites, and thus very highly connected in the reaction network.

We can evaluate our hypotheses with some clever addition of new reactions to resuscitate the cell. If (1) is true, then if we provide a free source of thiamine pyrophosphate, the cell should grow again:

```
--> THIAMINE-PYROPHOSPHATE[c]
```

Similarly if (2) is true, then if we provide a free sink for thiamine, the cell should grow again:

```
THIAMINE[c] -->
```

COBRApy provides several useful functions to create such reactions quickly - refer to the docs for more information.

In [40]:
# Hypothesis 1 : not enough THIAMINE-PYROPHOSPHATE[c] production

with model:
    model.reactions.get_by_id('EX_glc').bounds = (-5.44, 0)

    # Baseline growth rate is 0, with tptase knocked out
    tptase.knock_out()
    sol = model.optimize()
    print(f"Growth rate with THIAMIN-PYROPHOSPHOKINASE-RXN: {sol.objective_value:.4f} 1/hr\n")

    # Does adding free thiamine pyrophosphate fix things?
    # Add a new reaction using add_boundary - note that boundary type "sink" is, confusingly,
    # defined as a reversible reaction with the metabolite of interest on the left side and nothing on the right side,
    # so if we set the lower bound to < 0, we can produce the metabolite for free (i.e., contrary to the naming
    # convention, it can be either a source or a sink).
    source = model.add_boundary(
        model.metabolites.get_by_id("THIAMINE-PYROPHOSPHATE[c]"),
        type="sink",
        reaction_id="Thiamine_pyrophosphate_source",
        lb=-0.00012665498825421725,  # Set to exactly the lost flux from above!
        ub=0
    )
    print(f"New reaction: {source.id}\n\t{source.reaction}\t\t{source.bounds}")

    sol = model.optimize()
    print(f"Growth rate with new reaction: {sol.objective_value:.2f} 1/hr")


Growth rate with THIAMIN-PYROPHOSPHOKINASE-RXN: 0.0000 1/hr

New reaction: Thiamine_pyrophosphate_source
	THIAMINE-PYROPHOSPHATE[c] <-- 		(-0.00012665498825421725, 0)
Growth rate with new reaction: 0.57 1/hr


In [70]:
# Hypothesis 2 : not enough THIAMINE[c] consumption

with model:
    model.reactions.get_by_id('EX_glc').bounds = (-5.44, 0)

    # Baseline growth rate is 0, with tptase knocked out
    tptase.knock_out()
    sol = model.optimize()
    print(f"Growth rate with THIAMIN-PYROPHOSPHOKINASE-RXN: {sol.objective_value:.4f} 1/hr\n")

    # Does adding a thiamine sink fix things?
    source = model.add_boundary(
        model.metabolites.get_by_id("THIAMINE[c]"),
        type="sink",
        reaction_id="Thiamine_sink",
        lb=0,
        ub=0.00012665498825421725  # Set to exactly the lost flux from above!
    )
    print(f"New reaction: {source.id}\n\t{source.reaction}\t\t{source.bounds}")

    sol = model.optimize()
    print(f"Growth rate with new reaction: {sol.objective_value:.2f} 1/hr")


Growth rate with THIAMIN-PYROPHOSPHOKINASE-RXN: 0.0000 1/hr

New reaction: Thiamine_sink
	THIAMINE[c] --> 		(0, 0.00012665498825421725)
Growth rate with new reaction: 0.00 1/hr


In this case, adding a thiamine pyrophosphate source revives the model, while adding a thiamine sink does not. It's clear then that the issue is we're not producing enough thiamine pyrophosphate, and if this were truly a reaction we wanted to remove, we could now go looking for either (1) alternative sources of thiamine pyrophosphate, or (2) alternative ways to provide its downstream function in other reactions!

Note that while in this case, hypothesis 1 was true while hypothesis 2 was false, sometimes two things can be true at the same time... :O